<a href="https://colab.research.google.com/github/MuhammadNurilHuda/DeepFake-Detection/blob/main/Deepfake_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEEPFAKE DETECTION

In [ ]:
import os
import zipfile

local_zip = '/content/drive/MyDrive/Kaggle/deepfake-detection-challenge.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()
zip_ref.close()

In [ ]:
TRAIN_DIR = '/content/train_sample_videos/'

BATCH_SIZE = 1
SCALE = 0.25
N_FRAMES = None

In [ ]:
!pip install facenet-pytorch

     |████████████████████████████████| 1.9 MB 4.1 MB/s 


In [ ]:
!pip install efficientnet

     |████████████████████████████████| 50 kB 2.6 MB/s 


## Import Library

In [ ]:
from facenet_pytorch.models.inception_resnet_v1 import get_torch_home
torch_home = get_torch_home()

In [ ]:
import os
import glob
import json
import torch
import cv2
from PIL import Image
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from facenet_pytorch import MTCNN, InceptionResnetV1

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Running on device: {device}')

Running on device: cuda:0


## Define class

In [ ]:
class DetectionPipeline:
    """Pipeline class for detecting faces in the frames of a video file."""
    
    def __init__(self, detector, n_frames=None, batch_size=60, resize=None):
        """Constructor for DetectionPipeline class.
        
        Keyword Arguments:
            n_frames {int} -- Total number of frames to load. These will be evenly spaced
                throughout the video. If not specified (i.e., None), all frames will be loaded.
                (default: {None})
            batch_size {int} -- Batch size to use with MTCNN face detector. (default: {32})
            resize {float} -- Fraction by which to resize frames from original prior to face
                detection. A value less than 1 results in downsampling and a value greater than
                1 result in upsampling. (default: {None})
        """
        self.detector = detector
        self.n_frames = n_frames
        self.batch_size = batch_size
        self.resize = resize
    
    def __call__(self, filename):
        """Load frames from an MP4 video and detect faces.

        Arguments:
            filename {str} -- Path to video.
        """
        # Video reader dan panjang video
        v_cap = cv2.VideoCapture(filename)
        v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # mengambil 'n_frames' bingkai dengan jarak yang sama untuk sampel
        if self.n_frames is None:
            sample = np.arange(0, v_len)
        else:
            sample = np.linspace(0, v_len - 1, self.n_frames).astype(int)

        # Loop per frame
        faces = []
        frames = []
        for j in range(v_len):
            success = v_cap.grab()
            if j in sample:
                # Load frame
                success, frame = v_cap.retrieve()
                if not success:
                    continue
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = Image.fromarray(frame)
                
                # Resize frame 
                if self.resize is not None:
                    frame = frame.resize([int(d * self.resize) for d in frame.size])
                frames.append(frame)

                # Deteksi wajah dan reset frame list ketika batch full
                if len(frames) % self.batch_size == 0 or j == sample[-1]:
                    faces.extend(self.detector(frames))
                    frames = []

        v_cap.release()

        return faces

## Define Helper Function

In [ ]:
def process_faces(faces, feature_extractor):
    # Filter frame tanpa wajah
    faces = [f for f in faces if f is not None]
    if len(faces) == 0:
        return None
    faces = torch.cat(faces).to(device)

    # Generate facial feature vectors menggunakan pretrained model
    embeddings = feature_extractor(faces)

    # Hitung centroid untuk video dan jarak setiap vektor fitur wajah dari centroid
    centroid = embeddings.mean(dim=0)
    x = (embeddings - centroid).norm(dim=1).cpu().numpy()
    
    return x

## Data Preparation

In [ ]:
# Load Face Detector MTCNN
face_detector = MTCNN(margin=14, keep_all=True, factor=0.5, device=device).eval()

In [ ]:
# Load facial recognition RESNET
feature_extractor = InceptionResnetV1(pretrained='vggface2', device=device).eval()

  0%|          | 0.00/107M [00:00<?, ?B/s]

In [ ]:
# Face detection pipeline
detection_pipeline = DetectionPipeline(detector=face_detector, n_frames=N_FRAMES, batch_size=BATCH_SIZE, resize=SCALE)

In [ ]:
all_train_videos = glob.glob(os.path.join(TRAIN_DIR, '*.mp4'))
metadata_path = TRAIN_DIR + 'metadata.json'

with open(metadata_path, 'r') as f:
  metadata = json.load(f)